## 情報
### データの内容
- Store - the store number 　　　　　　　　　　　　　　　　　　　　　　店番号
- Dept - the department number　　　　　　　　　　　　　　　　　　　　　部門番号
- Date - the week　　　　　　　　　　　　　　　　　　　　　　　　　　　一週間単位の金曜日
- Weekly_Sales -  sales for the given department in the given store　　指定された店舗の指定部門の売上
- IsHoliday - whether the week is a special holiday week　　　　　　　　週が特別休暇の週であるかどうか
### ファイルの内容
- features.csv.zip
    - このファイルには、指定した日付の店舗、部署、および地域活動に関連する追加データが含まれています。以下のフィールドがあります。
        - Store - the store number
        - Date - the week
        - Temperature - average temperature in the region
        - Fuel_Price - cost of fuel in the region
        - MarkDown1-5 - anonymized data related to promotional markdowns that Walmart is running. MarkDown data is only available after Nov 2011, and is not available for all stores all the time. Any missing value is marked with an NA.
        - CPI - the consumer price index ※消費者物価指数
        - Unemployment - the unemployment rate
        - IsHoliday - whether the week is a special holiday week
- stores.csv
    - このファイルには、ストアのタイプとサイズを示す45のストアに関する匿名の情報が含まれています。
- train.csv.zip
    - これは2010-02-05から2012-11-01までの履歴トレーニングデータです。
- test.csv.zip
    - このファイルはtrain.csvと同じですが、週ごとの販売を控えています。このファイルの店舗、部署、日付の各トリプレットの売上を予測する必要があります。

### 備考
便宜上、4つの祝日はデータセット内の次の週に分類されます（すべての祝日がデータに含まれるわけではありません）。

- スーパーボウル：12-Feb-10,11-Feb-11,10-Feb-12,8-Feb-13
- 労働者の日：10-Sep-10,9-Sep-11,7-Sep-12,6-Sep-13
- 感謝祭：26-11月-10,25-11月-11日、23-11月-12日、29-11月-13日
- クリスマス：31-Dec-10、30-Dec-11、28-Dec-12、27-Dec-13

## ライブラリのインポート

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## データの読み込み
### チェックすべき項目
1. 学習データとテストデータの量は？
2. ターゲットの変数は何か？そのターゲットの変数は連続値？フラグ？
3. 説明変数は数値？カテゴリ変数？

In [2]:
train = pd.read_csv("datasets/train.csv.zip")
test = pd.read_csv("datasets/test.csv.zip")
stores = pd.read_csv("datasets/stores.csv")
features = pd.read_csv("datasets/features.csv.zip")
print("Train shape", train.shape)
print("Test shape", test.shape)
print("stores shape", stores.shape)
print("features shape", features.shape)

Train shape (421570, 5)
Test shape (115064, 4)
stores shape (45, 3)
features shape (8190, 12)


## アイデア

### WIP
- 2011年11月からしかないMarkDownの扱いはどうするか?また、CPIやUnemployment等も欠損値がある
    - 中央値で補完する
    - 欠損値はそうであるという旨の変数を新たに作る
        - データがある
        - 欠損している
        - もともとデータが無い時期である(MarkDownとCPIとUnemployment)
- IsHolidayの扱いは?
- Dateの扱いは?
    - 年、月、週数　等にバラす
- 学習モデルのパラメーターをチューニングする
    
### Done
- storesのデータをtrainとtestにくっつける
- featuresのTemperatureとFuel_price等の値をtrainとtestにくっつける
- 外れ値の検出方法は?
    - 基本的には中央値で対応する。詳細は別タスクで実施

### Reject
- featuresのTempertureは華氏から摂氏に変換

## trainとtestにstoresのデータを結合する

In [3]:
train = pd.merge(train, stores)
test = pd.merge(test, stores)

## featuresのTemperatureとFuel_price等の値をtrainとtestに結合させる

In [4]:
train = pd.merge(train, features)
test = pd.merge(test, features)

In [5]:
train = train.sort_values(by=["Store", "Dept", "Date"], ascending=True)
test = test.sort_values(by=["Store", "Dept", "Date"], ascending=True)

## カテゴリカル変数を整数にしてみる

In [6]:
# sotresのTypeは3種類しかない
stores['Type'].value_counts()

A    22
B    17
C     6
Name: Type, dtype: int64

In [7]:
# 【A -> 1, B -> 2, C -> 3】 に変換する
train['Type'] = train['Type'].mask(train['Type'] == 'A', 1)
train['Type'] = train['Type'].mask(train['Type'] == 'B', 2)
train['Type'] = train['Type'].mask(train['Type'] == 'C', 3)
test['Type'] = test['Type'].mask(test['Type'] == 'A', 1)
test['Type'] = test['Type'].mask(test['Type'] == 'B', 2)
test['Type'] = test['Type'].mask(test['Type'] == 'C', 3)

In [8]:
# featuresのIsHolidayは2種類しかない
features['IsHoliday'].value_counts()

False    7605
True      585
Name: IsHoliday, dtype: int64

In [9]:
train['IsHoliday'] = train['IsHoliday'].mask(train['IsHoliday'] == True, 1)
train['IsHoliday'] = train['IsHoliday'].mask(train['IsHoliday'] == False, 2)
test['IsHoliday'] = test['IsHoliday'].mask(test['IsHoliday'] == True, 1)
test['IsHoliday'] = test['IsHoliday'].mask(test['IsHoliday'] == False, 2)

In [10]:
test.head()

,Store,Dept,Date,IsHoliday,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment
0,1,1,2012-11-02,2,1,151315,55.32,3.386,6766.44,5147.70,50.82,3639.90,2737.42,223.462779,6.573
71,1,1,2012-11-09,2,1,151315,61.24,3.314,11421.32,3370.89,40.28,4646.79,6154.16,223.481307,6.573
142,1,1,2012-11-16,2,1,151315,52.92,3.252,9696.28,292.10,103.78,1133.15,6612.69,223.512911,6.573
213,1,1,2012-11-23,1,1,151315,56.23,3.211,883.59,4.17,74910.32,209.91,303.32,223.561947,6.573
285,1,1,2012-11-30,2,1,151315,52.34,3.207,2460.03,NaN,3838.35,150.57,6966.34,223.610984,6.573


## キー名の結合
#### Store_Dept_Date にする ex:Store = 2, Dept = 29, Date = 2013-06-07 -> 2_29_2013-06-07

In [11]:
train['PK_ID'] = train[['Store', 'Dept', 'Date']].apply(lambda x: '{}_{}_{}'.format(x[0], x[1], x[2]), axis=1)
test['PK_ID'] = test[['Store', 'Dept', 'Date']].apply(lambda x: '{}_{}_{}'.format(x[0], x[1], x[2]), axis=1)

In [12]:
train.head(1)

,Store,Dept,Date,Weekly_Sales,IsHoliday,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,PK_ID
0,1,1,2010-02-05,24924.5,2,1,151315,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,1_1_2010-02-05


In [13]:
test.head(1)

,Store,Dept,Date,IsHoliday,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,PK_ID
0,1,1,2012-11-02,2,1,151315,55.32,3.386,6766.44,5147.7,50.82,3639.9,2737.42,223.462779,6.573,1_1_2012-11-02


## trainとtestをくっつけて特徴量を作る

* testにはターゲット変数になる'revenue'がないから追加する（ただし全て0で)。
* また、後でtrainから来たデータなのか、testから来たデータなのかの見分けがつくようにis_trainでフラグを立てておく

In [14]:
test['Weekly_Sales']  = 0.0
test['is_train'] = 0
train['is_train'] = 1
whole_data = train.append(test)

In [24]:
whole_data.head()

,CPI,Date,Dept,Fuel_Price,IsHoliday,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,PK_ID,Size,Store,Temperature,Type,Unemployment,Weekly_Sales,is_train,Missing_val_Unemployment
0,211.096358,2010-02-05,1,2.572,2,NaN,NaN,NaN,NaN,NaN,1_1_2010-02-05,151315,1,42.31,1,8.106,24924.50,1,1.0
73,211.242170,2010-02-12,1,2.548,1,NaN,NaN,NaN,NaN,NaN,1_1_2010-02-12,151315,1,38.51,1,8.106,46039.49,1,1.0
145,211.289143,2010-02-19,1,2.514,2,NaN,NaN,NaN,NaN,NaN,1_1_2010-02-19,151315,1,39.93,1,8.106,41595.55,1,1.0
218,211.319643,2010-02-26,1,2.561,2,NaN,NaN,NaN,NaN,NaN,1_1_2010-02-26,151315,1,46.63,1,8.106,19403.54,1,1.0
290,211.350143,2010-03-05,1,2.625,2,NaN,NaN,NaN,NaN,NaN,1_1_2010-03-05,151315,1,46.50,1,8.106,21827.90,1,1.0


説明変数(predictor variables)とターゲット変数(target variable)、また説明変数から除外する変数を定義する。

In [17]:
whole_data.columns

Index(['CPI', 'Date', 'Dept', 'Fuel_Price', 'IsHoliday', 'MarkDown1',
       'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'PK_ID', 'Size',
       'Store', 'Temperature', 'Type', 'Unemployment', 'Weekly_Sales',
       'is_train'],
      dtype='object')

## 欠損値の対処を行う

In [23]:
whole_data['Unemployment'].median()

7.795

In [22]:
# 確認用
whole_data[ (whole_data['Store'] == 1) & (whole_data['Date'] == '2013-05-03') ].head(3)

,CPI,Date,Dept,Fuel_Price,IsHoliday,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,PK_ID,Size,Store,Temperature,Type,Unemployment,Weekly_Sales,is_train,Missing_val_Unemployment
1864,182.44242,2013-05-03,1,3.386,2,2298.63,2.0,129.9,55.46,1301.04,1_1_2013-05-03,151315,1,66.66,1,7.795,0.0,0,2.0
1865,182.44242,2013-05-03,2,3.386,2,2298.63,2.0,129.9,55.46,1301.04,1_2_2013-05-03,151315,1,66.66,1,7.795,0.0,0,2.0
1866,182.44242,2013-05-03,3,3.386,2,2298.63,2.0,129.9,55.46,1301.04,1_3_2013-05-03,151315,1,66.66,1,7.795,0.0,0,2.0


#### UnemploymentとCPIの欠損値ありなしフラグを付与する

In [20]:
# UnemploymentとCPIのNaNのレコードにそれぞれMissing_val_UnemploymentとMissing_val_CPI列を作成する
whole_data['Missing_val_Unemployment'] = 1.0
whole_data['Missing_val_Unemployment'] = whole_data['Missing_val_Unemployment'].mask(pd.isnull(whole_data['Unemployment']), 2)

In [21]:
# UnemploymentのNaNを中央値に変換する
unemployment_median = whole_data['Unemployment'].median()
whole_data['Unemployment'] = whole_data['Unemployment'].fillna(unemployment_median)
# CPIのNaNを中央値に変換する
CPI_median = whole_data['CPI'].median()
whole_data['CPI'] = whole_data['CPI'].fillna(CPI_median)

In [25]:
exclude_colnames = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'PK_ID', 'Date', 'Weekly_Sales']
target_colnames = 'Weekly_Sales'
predictor_colnames = [colname for colname in whole_data.columns if colname not in exclude_colnames]

In [26]:
predictor_colnames

['CPI',
 'Dept',
 'Fuel_Price',
 'IsHoliday',
 'Size',
 'Store',
 'Temperature',
 'Type',
 'Unemployment',
 'is_train',
 'Missing_val_Unemployment']

In [27]:
whole_X_df = whole_data[predictor_colnames]
whole_y_df = whole_data[target_colnames]

## カテゴリカル変数で整数にしていないものをダミー変数に変換する

In [28]:
# whole_X_df = pd.get_dummies(whole_X_df, columns=['City'])

## 以下は学習実行

In [29]:
feature_names = whole_X_df.columns

In [30]:
feature_names

Index(['CPI', 'Dept', 'Fuel_Price', 'IsHoliday', 'Size', 'Store',
       'Temperature', 'Type', 'Unemployment', 'is_train',
       'Missing_val_Unemployment'],
      dtype='object')

In [31]:
X_train = np.array(whole_X_df[whole_X_df['is_train']==1])
y_train = np.array(whole_y_df[whole_X_df['is_train']==1])
X_test = np.array(whole_X_df[whole_X_df['is_train']==0])
y_test = np.array(whole_y_df[whole_X_df['is_train']==0])

In [32]:
X_test

array([[223.4627793, 1, 3.386, ..., 6.5729999999999995, 0, 1.0],
       [223.48130730000003, 1, 3.3139999999999996, ..., 6.5729999999999995,
        0, 1.0],
       [223.5129105, 1, 3.252, ..., 6.5729999999999995, 0, 1.0],
       ..., 
       [182.4424199, 98, 3.614, ..., 7.795, 0, 2.0],
       [182.4424199, 98, 3.737, ..., 7.795, 0, 2.0],
       [182.4424199, 98, 3.804, ..., 7.795, 0, 2.0]], dtype=object)

## モデリング - クロスバリデーション

In [33]:
from sklearn.linear_model import Ridge
from sklearn.cross_validation import KFold
from sklearn.metrics import mean_squared_error

In [34]:
kf = KFold(n=X_train.shape[0], n_folds=7, random_state=1234, shuffle=True)

In [35]:
kf

sklearn.cross_validation.KFold(n=421570, n_folds=7, shuffle=True, random_state=1234)

In [36]:
clf = Ridge()

In [37]:
clf

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [38]:
rmse_cv_scores = []
for train_index, test_index in kf:
    clf.fit(X_train[train_index, :], y_train[train_index])
    ypred = clf.predict(X_train[test_index])
    rmse = mean_squared_error(y_train[test_index], ypred)**0.5
    rmse_cv_scores.append(rmse)
    print('rmse: ', rmse)
print('Mean RMSE: %f'% np.mean(rmse_cv_scores))
print('Std RMSE: +-', np.std(rmse_cv_scores))

rmse:  21974.6950066
rmse:  21496.37024
rmse:  21784.6228713
rmse:  21780.7564974
rmse:  21332.5882982
rmse:  21799.1715499
rmse:  21809.2959337
Mean RMSE: 21711.071485
Std RMSE: +- 202.314811102


In [39]:
clf.fit(X_train, y_train)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

## submissionファイルを作成

In [40]:
ypred_test = clf.predict(X_test)

In [41]:
submission = test[['PK_ID', 'Weekly_Sales']].copy()

In [42]:
submission.loc[: , 'Weekly_Sales'] = ypred_test

In [43]:
submission.columns = ['Id', 'Weekly_Sales']

In [44]:
# Make submission File 
submission.to_csv("submission_1.csv", index=False)

## submission 記録

- テスト用の最初の1回目 (3/11 15時頃)
    - Mean RMSE: 22043.852869
    - Std RMSE: +- 202.137520087
    - Score: 20040.44485

- Unemployment と CPIのNaNを単純に中央値で埋めたもの (3/12 1時頃)
    - Mean RMSE: 21711.071485
    - Std RMSE: +- 202.314811102
    - Score: 19537.64481

- Unemploymentの欠損値だったフラグ「Missing_val_Unemployment」を付与
    - Mean RMSE: 21711.071485
    - Std RMSE: +- 202.314811102
    - Score: 19537.64481

## whole_dataの列の順番指定をするサンプル

In [51]:
whole_data.head(1)

,CPI,Date,Dept,Fuel_Price,IsHoliday,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,PK_ID,Size,Store,Temperature,Type,Unemployment,Weekly_Sales,is_train,Missing_val_Unemployment
0,211.096358,2010-02-05,1,2.572,2,NaN,NaN,NaN,NaN,NaN,1_1_2010-02-05,151315,1,42.31,1,8.106,24924.5,1,1.0


In [ ]:
whole_data.ix[:,['Store','Dept','Date']]